In [7]:
import os
import glob

In [1]:
import findspark
findspark.init()

IndexError: list index out of range

In [5]:
spark_python = os.path.join("C:\Applications\spark-2.4.5-bin-hadoop2.7", 'python')

In [11]:
os.path.join(spark_python, 'lib', 'py4j-*.zip')

'C:\\Applications\\spark-2.4.5-bin-hadoop2.7\\python\\lib\\py4j-*.zip'

In [ ]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [ ]:
file_dir = "processed_tweet_price.csv"
#file_dir = "hdfs://localhost:8020/data/processed_tweet_price.csv"

In [ ]:
df = spark.read.csv(file_dir, header="true", inferSchema="true")

In [ ]:
df.cache()

In [ ]:
df.printSchema()

In [ ]:
df = df.select([col(c).cast("double").alias(c) for c in df.columns])

In [ ]:
train, test = df.randomSplit([0.7, 0.3])